# Import libraries

In [1]:
import pandas as pd
import numpy as np
import copy, random, math
import ast
import folium

from geopy.distance import geodesic
from folium.plugins import MarkerCluster

# Loading data & Data preparation

In [2]:
distances = pd.read_csv('distances.csv')
establishments = pd.read_csv('establishments.csv')

# shift the column names to the left
distances.columns = pd.Index(list(distances.columns[1:]) + ['p_1000'])

very_small_establishments = establishments.iloc[0:21,:]
very_small_distances = distances.iloc[0:21,0:21]

small_establishments = establishments.iloc[0:101,:]
small_distances = distances.iloc[0:101,0:101]

medium_establishments = establishments.iloc[0:501,:]
medium_distances = distances.iloc[0:501,0:501]

large_establishments = establishments
large_distances = distances

# Problem definition

In [5]:
# convert the DataFrame to a dictionary
traveling_times = {}
for i in range(len(very_small_distances)):
    traveling_times[i] = very_small_distances.iloc[i].tolist()

num_establishments = len(traveling_times)-1
vehicles = int(num_establishments*0.1)
inspection_times = very_small_establishments[['Inspection Time']]
open_hours = very_small_establishments[['Opening Hours']]


def generate_random_solution():
    return np.random.randint(1, vehicles + 1, num_establishments)

def objective_function(chromosome, traveling_times, inspection_times, open_hours):
    total_travel_time = 0
    total_inspection_time = 0
    total_time_per_route = [0 for i in range(max(chromosome))]
    j = -1

    # Initialize an empty list to store the routes
    routes = [[] for i in range(max(chromosome))]
    best_routes = []

    # Assign each establishment to a route
    for i in range(len(chromosome)):
        routes[chromosome[i]-1].append(i+1)

    # Calculate the total travel time and inspection time for all routes
    for route in routes:
        if len(route) == 0:
            continue
        j += 1
        route_travel_time = 0
        route_inspection_time = 0
        current_time = 0
        # Add the departure/arrival establishment to the beginning and end of the route
        route = [0] + route + [0]
        
        # Use a greedy search algorithm to find the optimal path for the current route
        route = greedy_search(route, heuristic_distances)
        best_routes.append(route)
        
        start_time = ast.literal_eval(open_hours.iloc[route[1]][0]).index(1)  # Inspection start time of first establishment in route
        #print('start time: {}'.format(start_time))

        for i in range(1, len(route)):
            # Calculate traveling time between establishments
            current_establishment = route[i-1]
            next_establishment = route[i]
            travel_time = traveling_times[current_establishment][next_establishment]
            #route_travel_time += traveling_times[current_establishment][next_establishment]
            
            # Calculate inspection time and waiting time based on establishment's schedule
            current_inspection_time = inspection_times.iloc[next_establishment][0]*60 # Convert minutes to seconds
            current_open_hours = ast.literal_eval(open_hours.iloc[next_establishment][0])
            
            # Calculate waiting time if establishment is closed
            if i == 1:
                waiting_time = 0
            else:
                current_hour = int((current_time + travel_time + (start_time*3600))/3600)
                #print('Current hour: {}'.format(current_hour))
                while current_open_hours[current_hour % 24] == 0:
                    current_hour += 1
                waiting_time = (current_hour * 3600) - (current_time + travel_time + (start_time*3600))
                #print('({} * 3600) - ({} + {} + ({}*3600))'.format(current_hour, current_time, travel_time, start_time))
                
            # Add inspection time and waiting time to current time
            if i > 1:
                current_time += max(waiting_time, 0)
            current_time += travel_time
            current_time += current_inspection_time
            
            #print('Iteration {} ---> current establishment: {}, next establishment: {}, inspection time: {}, open hours array: {}, travel time: {}, waiting time: {}, current time: {}'.format(i, current_establishment, next_establishment, current_inspection_time, current_open_hours, travel_time, waiting_time, current_time))
            # Reset waiting time
            waiting_time = 0
            
            # Add inspection time and traveling time to route times
            #route_inspection_time += current_inspection_time
            route_travel_time = 0

        # Add total time for current route to total time per route
        total_time_per_route[j] = current_time
        #print('total time per route ----> {}'.format(total_time_per_route))

    # Add total time for all routes to total travel time
    total_travel_time += sum(total_time_per_route)
    
    return -total_travel_time, best_routes # Minimize the sum of inspection and travel time

def objective_function_a_star(chromosome, traveling_times, inspection_times, open_hours):
    
    total_travel_time = 0
    l = 1
    
    # Initialize an empty list to store the routes
    routes = [[] for i in range(max(chromosome))]
    best_routes = []
    
    # Assign each establishment to a route
    for i in range(len(chromosome)):
        routes[chromosome[i]-1].append(i+1)
    
    # Calculate the total travel time for all routes
    for route in routes:
        if len(route) == 0:
            continue
        j = 1
        current_time = 0
        
        # Create a set with all establishments in the route
        route_establishments = set(route)
        route_len = len(route_establishments)
        
        # Add the depot to the beginning of the route
        route = [0]
        
        # Initialize the route travel time as zero
        route_travel_time = 0
        
        while route_establishments:
            # Calculate the A* algorithm for the next establishment
            current_establishment = route[-1]
            next_establishment = None
            best_score = float('inf')
            for e in route_establishments:
                i = 1
                # Calculate the total time (travel time, inspection time, waiting time, heuristic value) for the next establishment
                travel_time = traveling_times[route[-1]][e]
                inspection_time = inspection_times.iloc[e][0] * 60
                current_open_hours = ast.literal_eval(open_hours.iloc[e][0])
                if len(route_establishments) == route_len:
                    start_time = current_open_hours.index(1)
                i += 1
                # Calculate waiting time if establishment is closed
                if len(route) == 1:
                    waiting_time = 0
                else:
                    current_hour = int((current_time + travel_time + (start_time*3600))/3600)
                    while current_open_hours[current_hour % 24] == 0:
                        current_hour += 1
                    waiting_time = (current_hour * 3600) - (current_time + travel_time + (start_time*3600))
                
                # Add inspection time, waiting time, and travel time to the current time
                time_to_next = max(waiting_time, 0) + inspection_time + travel_time
                total_time = current_time + time_to_next
                heuristic_value = heuristic_distances[e][0]
                score = total_time + heuristic_value
                
                # If the current establishment has the smallest score, update next_establishment
                if score < best_score:
                    '''
                    best_waiting_time = max(waiting_time, 0)
                    best_open_hours = current_open_hours
                    best_inspection_time = inspection_time
                    if len(route) == 1:
                        best_start_time = start_time
                    else:
                        best_current_hour = current_hour
                    best_current_time = total_time
                    best_travel_time = travel_time
                    '''
                    time_to_next_establishment = time_to_next
                    next_establishment = e
                    best_score = score
            '''
            if len(route) > 1:
                print('Current hour: {}'.format(best_current_hour))
                print('({} * 3600) - ({} + {} + ({}*3600))'.format(best_current_hour, current_time, best_travel_time, best_start_time))
            else:
                print('start time: {}'.format(best_start_time))
            '''
            
            # Add next establishment to route and remove from set
            route_establishments.remove(next_establishment)
            route.append(next_establishment)
            
            #print('Update current time: {} + {}'.format(current_time, time_to_next_establishment))
            # Update the current time and route travel time
            current_time += time_to_next_establishment
            #route_travel_time += travel_time
            #print('Iteration {} ---> current establishment: {}, next establishment: {}, next inspection time: {}, open hours array: {}, travel time: {}, waiting time: {}, current time: {}'.format(j, current_establishment, next_establishment, best_inspection_time, best_open_hours, best_travel_time, best_waiting_time, best_current_time))
        
            j += 1
        # Add the depot to the end of the route
        route.append(0)
        best_routes.append(route)
        #print('Route: {}'.format(route))
        # Calculate the total time for the current route and add to total time per route
        total_time_per_route = current_time + traveling_times[route[-2]][0]
        #print('Traveling time back to the depot: {}'.format(traveling_times[route[-2]][0]))
        #print('Total travel time in route {}: {}'.format(l, total_time_per_route))
        total_travel_time += total_time_per_route
        l += 1
        
    return -total_travel_time, best_routes # Minimize the total travel time

def geodesic_time_matrix(establishments, speed_kmph):
    """
    Calculates the travel time matrix between all combinations of establishments based on their
    latitude and longitude coordinates, assuming a given speed in km/h.
    
    Parameters:
    establishments (pandas.DataFrame): A DataFrame containing the latitude and longitude coordinates of all establishments.
    speed_kmph (float): The speed of travel in km/h.
    
    Returns:
    pandas.DataFrame: A DataFrame containing the travel time matrix between all combinations of establishments in seconds.
    """
    # Create a 2D array of establishment coordinates
    coordinates = establishments[['Latitude', 'Longitude']].values
    
    # Initialize an empty time matrix with the same shape as the coordinates matrix
    time_matrix = pd.DataFrame(index=establishments.index, columns=establishments.index)
    
    # Calculate pairwise travel times between all establishments
    for i in range(len(coordinates)):
        for j in range(len(coordinates)):
            # Calculate the distance between establishments i and j using geodesic distance
            distance_km = geodesic(coordinates[i], coordinates[j]).km
            
            # Calculate the travel time in seconds using the speed in km/h
            travel_time_sec = (distance_km / speed_kmph) * 3600
            
            # Set the travel time in the time matrix
            time_matrix.iloc[i, j] = travel_time_sec
    
    return time_matrix

def greedy_search(route, heuristic):
    current_node = 0
    visited = [False] * (max(route) + 1)
    visited[current_node] = True
    path = [current_node]
    while len(path) < len(route):
        next_node = -1
        min_distance = float('inf')
        for i in range(len(route)):
            if not visited[route[i]]:
                distance = heuristic[route[i]][route[0]]
                if distance < min_distance:
                    next_node = route[i]
                    min_distance = distance
        visited[next_node] = True
        path.append(next_node)
        current_node = next_node
    path.pop() # remove last element (should be 0)
    path.append(0)
    return path

def display_routes(routes):
    # Define the map center and zoom level
    center = [41.160304, -8.602478]
    zoom = 10.4

    # Create a map object
    tile = 'OpenStreetMap'
    map = folium.Map(location=center, zoom_start=zoom, tiles=tile)

    # Define the color palette for the routes
    route_colors = ['#d62728','#3388ff','#33cc33','#ff9933','#800080','#ff3399','#808080','#f5f5dc','#32174d','#ffffff',
              '#5f9ea0','#d3d3d3','#add8e6','#00008b','#90ee90','#006400','#8b0000','#ff6666']
    marker_colors = ['red', 'blue', 'green', 'orange', 'purple', 'pink', 'gray', 'beige', 'darkpurple', 'white', 
                     'cadetblue', 'lightgray', 'lightblue', 'darkblue', 'lightgreen', 'darkgreen', 'darkred', 'lightred']

    # Create checkboxes for each route
    checkboxes = []
    for i, route in enumerate(routes):
        label = f"Route {i+1}"
        checkbox = folium.FeatureGroup(name=label, overlay=True, control=True)
        for k, j in enumerate(route[1:-1]):
            folium.Marker(
                location=(establishments.iloc[j]['Latitude'], establishments.iloc[j]['Longitude']),
                icon=folium.Icon(color=marker_colors[i % len(route_colors)]),
                tooltip=f"Visit order: {k+1}",
                popup=f"{establishments.iloc[j]['Address']}",
            ).add_to(checkbox)
        folium.PolyLine(
            locations=[(establishments.iloc[j]['Latitude'], establishments.iloc[j]['Longitude']) for j in route],
            color=route_colors[i % len(route_colors)],
            popup=label,
        ).add_to(checkbox)
        checkboxes.append(checkbox)
    
    # Add caution marker to the depot
    folium.Marker(location=center, icon=folium.Icon(icon='home', prefix='fa', color='black'), tooltip='Depot', 
                 popup=f"{establishments.iloc[0]['Address']}").add_to(map)
    
    # Add the checkboxes to the map
    for checkbox in checkboxes:
        checkbox.add_to(map)
    
    # Add a layer control to the map
    folium.LayerControl().add_to(map)
    
    # Display the map
    return map

In [6]:
heuristic_distances = geodesic_time_matrix(establishments, 70)

# Subtract the heuristic distance matrix from the travel time matrix
difference_matrix = heuristic_distances - distances

# Check if there are any positive values in the difference matrix
if (difference_matrix > 0).any().any():
    print('There is at least one travel time that is lower than the heuristic value!')
else:
    print('All travel times are greater than or equal to the heuristic value.')

All travel times are greater than or equal to the heuristic value.


# Genetic Algorithm

In [7]:
def midpoint_crossover(solution_1, solution_2):
    mid_index = math.trunc(len(solution_1) / 2)
    child_1 = np.append(solution_1[0:mid_index], solution_2[mid_index:]) 
    child_2 = np.append(solution_2[0:mid_index], solution_1[mid_index:])
    return child_1, child_2

def randompoint_crossover(solution_1, solution_2):
    length = len(solution_1)
    crossover_point = random.randint(1, length - 1)
    child_1 = np.concatenate([solution_1[:crossover_point], solution_2[crossover_point:]])
    child_2 = np.concatenate([solution_2[:crossover_point], solution_1[crossover_point:]])
    return child_1, child_2

def uniform_crossover(solution_1, solution_2):
    """
    Performs uniform crossover (UX) on two solutions.
    
    Args:
        solution_1 (list): The first solution.
        solution_2 (list): The second solution.
        
    Returns:
        Two new solutions created by randomly swapping genes between solution_1 and solution_2.
    """
    # Initialize children as copies of the parent solutions
    child_1 = solution_1.copy()
    child_2 = solution_2.copy()

    # Determine the genes to swap
    genes_to_swap = np.random.choice(len(solution_1), len(solution_1), replace=False)

    # Swap the genes between the two children
    for i in genes_to_swap:
        if child_1[i] != child_2[i]:
            child_1[i], child_2[i] = child_2[i], child_1[i]

    return child_1, child_2

def mutate_solution_1(solution):
    index_1 = np.random.randint(0, len(solution))
    index_2 = (index_1 + np.random.randint(0, len(solution))) % (len(solution) - 1) # Efficient way to generate a non-repeated index
    solution[index_1], solution[index_2] = solution[index_2], solution[index_1]
    return solution

def mutate_solution_2(solution):
    index = np.random.randint(0, len(solution))
    solution[index] = np.random.randint(1, vehicles + 1)
    return solution

def generate_population(population_size):
    solutions = []
    for i in range(population_size):
        solutions.append(generate_random_solution())
    return solutions

def print_population(population, objective_func):
    solutions = []
    for i in range(len(population)):
        print(f"Solution {i+1}: {population[i]}, {-objective_func(population[i], traveling_times, inspection_times, open_hours)[0]}")
        
def tournament_select(population, tournament_size, objective_func):
    population_copy = copy.deepcopy(population)
    best_solution = population_copy[0]
    best_score = objective_func(population_copy[0], traveling_times, inspection_times, open_hours)[0]
    for i in range(tournament_size):
        index = np.random.randint(0, len(population_copy))
        score = objective_func(population_copy[index], traveling_times, inspection_times, open_hours)[0]
        if score > best_score:
            best_score = score
            best_solution = population_copy[index]
        del population_copy[index]
    return best_solution

def get_greatest_fit(population, objective_func):
    best_solution = population[0]
    best_score, best_routes = objective_func(population[0], traveling_times, inspection_times, open_hours)
    for i in range(1, len(population)):
        score, routes = objective_func(population[i], traveling_times, inspection_times, open_hours)
        if score > best_score:
            best_score = score
            best_solution = population[i]
            best_routes = routes
    return best_solution, best_score, best_routes

def replace_least_fittest(population, offspring, objective_func):
    least_fittest_index = 0
    least_fittest_value = objective_func(population[0], traveling_times, inspection_times, open_hours)[0]
    for i in range(1, len(population)):
        score = objective_func(population[i], traveling_times, inspection_times, open_hours)[0]
        if score < least_fittest_value:
            least_fittest_value = score
            least_fittest_index = i
    population[least_fittest_index] = offspring

def roulette_select(population, objective_func):
    score_sum = sum([objective_func(solution, traveling_times, inspection_times, open_hours)[0] for solution in population])
    selection_probabilities = [objective_func(solution, traveling_times, inspection_times, open_hours)[0] / score_sum for solution in population]
    return population[np.random.choice(len(population), p=selection_probabilities)]

In [8]:
#Test Crossover
s1 = generate_random_solution()
s2 = generate_random_solution()
print('solution 1 -----> {}'.format(s1))
print('solution 2 -----> {}'.format(s2))
c1, c2 = midpoint_crossover(s1, s2)
print('child 1 -----> {}'.format(c1))
print('child 2 -----> {}'.format(c2))
c1, c2 = randompoint_crossover(s1, s2)
print('new child 1 -----> {}'.format(c1))
print('new child 2 -----> {}'.format(c2))
c1, c2 = uniform_crossover(s1, s2)
print('new child 1 ux -----> {}'.format(c1))
print('new child 2 ux -----> {}'.format(c2))
#Test Mutation
c3 = mutate_solution_1(c1)
c4 = mutate_solution_1(c2)
print('mutation of child 1 -----> {}'.format(c3))
print('mutation of child 2 -----> {}'.format(c4))

solution 1 -----> [2 1 2 2 2 1 2 2 1 2 1 2 2 1 2 1 2 2 2 2]
solution 2 -----> [1 2 2 1 2 2 1 2 1 2 1 1 2 2 1 2 1 1 1 2]
child 1 -----> [2 1 2 2 2 1 2 2 1 2 1 1 2 2 1 2 1 1 1 2]
child 2 -----> [1 2 2 1 2 2 1 2 1 2 1 2 2 1 2 1 2 2 2 2]
new child 1 -----> [2 1 2 2 2 2 1 2 1 2 1 1 2 2 1 2 1 1 1 2]
new child 2 -----> [1 2 2 1 2 1 2 2 1 2 1 2 2 1 2 1 2 2 2 2]
new child 1 ux -----> [1 2 2 1 2 2 1 2 1 2 1 1 2 2 1 2 1 1 1 2]
new child 2 ux -----> [2 1 2 2 2 1 2 2 1 2 1 2 2 1 2 1 2 2 2 2]
mutation of child 1 -----> [1 2 2 1 2 2 1 2 1 2 1 1 2 2 1 2 1 1 1 2]
mutation of child 2 -----> [2 1 2 2 2 1 2 2 1 2 1 2 2 1 2 1 2 2 2 2]


In [9]:
pop = generate_population(10)
print_population(pop, objective_function_a_star)

tournament = tournament_select(pop, 2, objective_function_a_star)
print('Tournament -----> {}'.format(tournament))

best_fit = get_greatest_fit(pop, objective_function_a_star)
print('Greatest fit -----> {}'.format(best_fit))

roulette = roulette_select(pop, objective_function_a_star)
print('Roulette -----> {}'.format(roulette))

Solution 1: [2 2 2 2 1 2 2 2 1 2 1 1 2 2 2 1 2 2 2 2], 134453.8
Solution 2: [1 1 2 1 1 1 1 2 1 1 1 2 1 2 2 2 2 2 2 2], 151590.0
Solution 3: [1 1 1 1 2 2 2 1 2 1 2 2 1 1 1 2 2 2 2 2], 159657.9
Solution 4: [2 2 1 2 2 2 1 1 2 2 1 1 2 1 2 2 2 2 1 2], 158607.5
Solution 5: [1 1 2 1 2 1 1 2 1 2 2 2 2 2 2 2 1 1 1 2], 159657.90000000002
Solution 6: [2 2 2 2 2 1 1 1 1 1 2 2 1 2 1 2 1 1 1 2], 124907.6
Solution 7: [1 1 2 2 2 1 2 2 1 2 2 2 2 2 1 2 2 2 1 2], 132429.0
Solution 8: [1 1 1 1 1 1 2 2 1 2 2 1 1 1 1 1 1 2 1 1], 140354.1
Solution 9: [1 1 1 1 2 1 2 2 2 2 1 1 2 2 1 1 1 1 2 1], 158134.6
Solution 10: [2 1 1 1 1 2 2 1 2 2 1 2 2 1 1 2 2 2 2 2], 145674.0
Tournament -----> [2 2 2 2 2 1 1 1 1 1 2 2 1 2 1 2 1 1 1 2]
Greatest fit -----> (array([2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 2, 2, 1, 2, 1, 2, 1, 1, 1, 2]), -124907.6, [[0, 19, 17, 6, 13, 15, 10, 18, 8, 7, 9, 0], [0, 5, 2, 16, 1, 4, 11, 14, 20, 12, 3, 0]])
Roulette -----> [1 1 1 1 2 2 2 1 2 1 2 2 1 1 1 2 2 2 2 2]


In [10]:
def genetic_algorithm(num_iterations, population_size, crossover_func, mutation_func, objective_func, log=False):
    population = generate_population(population_size)
    
    best_solution = population[0] # Initial solution
    best_score = objective_func(population[0], traveling_times, inspection_times, open_hours)[0]
    best_solution_generation = 0 # Generation on which the best solution was found
    
    generation_no = 0
    
    print(f"Initial solution: {best_solution}, score: {-best_score}")
    
    while(num_iterations > 0):
        
        generation_no += 1
        
        tournment_winner_sol = tournament_select(population, 4, objective_func)
        roulette_winner_sol = roulette_select(population, objective_func)
        
        # Next generation
        crossover_sol_1, crossover_sol_2 = crossover_func(tournment_winner_sol, roulette_winner_sol)
    
        if np.random.randint(0, 10) < 1: # 40% chance to perform mutation
            replace_least_fittest(population, mutation_func(crossover_sol_1), objective_func)
            replace_least_fittest(population, mutation_func(crossover_sol_2), objective_func)
        else:
            replace_least_fittest(population, crossover_sol_1, objective_func)
            replace_least_fittest(population, crossover_sol_2, objective_func)
        
        # Checking the greatest fit among the current population
        greatest_fit, greatest_fit_score, best_routes = get_greatest_fit(population, objective_func)
        if greatest_fit_score > best_score:
            best_solution = greatest_fit
            best_score = greatest_fit_score
            best_routes
            best_solution_generation = generation_no
            if log:
                print(f"\nGeneration: {generation_no }")
                print(f"Solution: score: {-best_score}")
                print_population(population, objective_func)
        else:
            num_iterations -= 1
        
    print(f"  Final solution: {best_solution}, score: {-best_score}, best routes: {best_routes}")
    print(f"  Found on generation {best_solution_generation}")
    
    return best_solution, best_routes

In [54]:
print("\nGeneticAlgorithm:\n")
solution, routes = genetic_algorithm(200, 30, randompoint_crossover, mutate_solution_1, objective_function_a_star, True)


GeneticAlgorithm:

Initial solution: [1 1 1 1 1 2 1 2 1 2 1 2 2 2 1 2 1 2 2 1], score: 159657.90000000002

Generation: 1
Solution: score: 120357.70000000001
Solution 1: [1 1 1 1 1 2 1 2 1 2 1 2 2 2 1 2 1 2 2 1], 159657.90000000002
Solution 2: [2 1 1 1 2 2 2 1 2 2 2 1 2 1 2 2 2 1 2 1], 146146.9
Solution 3: [1 1 1 1 1 2 2 2 1 1 2 2 2 2 1 1 1 2 1 1], 149415.1
Solution 4: [2 2 2 2 2 2 1 2 1 2 2 2 2 2 1 2 2 2 1 1], 158439.6
Solution 5: [1 2 1 2 2 1 2 2 1 2 2 2 2 2 2 1 2 1 1 2], 146388.2
Solution 6: [1 1 1 1 1 1 1 2 1 2 2 2 1 2 1 1 2 1 1 1], 128888.0
Solution 7: [2 1 1 1 1 2 1 2 1 1 1 2 2 1 2 1 2 2 1 1], 143066.2
Solution 8: [1 2 2 2 2 2 1 2 2 1 2 1 1 2 2 2 2 1 2 2], 120357.70000000001
Solution 9: [2 1 1 2 1 1 2 2 2 1 1 1 2 2 2 2 1 2 2 1], 170491.8
Solution 10: [1 1 1 1 1 1 1 2 1 2 1 1 1 2 1 1 1 1 2 1], 126709.6
Solution 11: [1 2 2 1 2 2 2 1 1 2 2 1 1 2 1 1 2 1 2 2], 153945.80000000002
Solution 12: [1 2 1 2 1 2 2 2 1 2 2 2 1 2 1 1 2 1 1 1], 135935.6
Solution 13: [1 1 2 2 2 2 1 1 1 1 1 2 1 2

  Final solution: [1 1 1 1 1 1 1 2 1 2 2 2 2 1 2 1 1 1 1 1], score: 111212.80000000002, best routes: [[0, 19, 17, 5, 2, 6, 16, 4, 1, 14, 20, 7, 18, 3, 9, 0], [0, 13, 11, 15, 10, 12, 8, 0]]
  Found on generation 42


In [53]:
print("\nGeneticAlgorithm:\n")
solution, routes = genetic_algorithm(200, 30, uniform_crossover, mutate_solution_1, objective_function, True)


GeneticAlgorithm:

Initial solution: [2 1 2 2 1 2 2 2 2 1 2 1 2 1 2 2 2 2 1 1], score: 256233.0

Generation: 1
Solution: score: 179564.1
Solution 1: [2 1 2 2 1 2 2 2 2 1 2 1 2 1 2 2 2 2 1 1], 256233.0
Solution 2: [2 2 2 1 2 2 1 1 1 1 1 1 1 2 2 2 1 1 1 2], 234915.3
Solution 3: [2 2 2 2 2 2 2 1 1 2 1 2 1 1 2 2 1 1 2 1], 198805.8
Solution 4: [2 1 2 1 2 2 2 2 2 1 1 2 2 2 1 2 1 1 1 1], 240792.2
Solution 5: [1 2 2 1 2 1 1 2 2 2 1 1 1 2 1 2 2 2 2 1], 238063.1
Solution 6: [1 1 2 2 2 2 2 2 1 2 2 1 1 2 2 1 2 2 2 2], 229648.30000000002
Solution 7: [2 2 1 1 1 1 2 2 2 2 2 2 2 2 1 1 2 2 2 1], 192795.39999999997
Solution 8: [2 2 2 1 1 2 1 2 1 2 2 1 2 2 2 1 1 2 1 1], 188525.90000000002
Solution 9: [2 2 2 1 2 1 2 2 2 2 2 2 2 2 2 2 2 2 1 1], 251684.19999999998
Solution 10: [1 2 2 2 1 2 2 1 1 2 1 1 2 1 1 2 2 1 2 1], 240065.40000000002
Solution 11: [1 1 1 2 1 2 2 2 2 1 1 2 2 2 1 2 2 1 2 2], 236872.60000000003
Solution 12: [1 1 2 1 2 1 2 2 2 1 2 2 2 2 2 1 2 1 1 1], 262879.0
Solution 13: [1 1 2 2 2 2 1 1 2

In [55]:
best_greedy = [1, 2, 1, 2, 1, 1, 1, 2, 1, 2, 2, 1, 2, 1, 1, 2, 2, 2, 1, 1]
best_a_star = [1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 2, 2, 2, 1, 2, 1, 1, 1, 1, 1]
score, routes = objective_function(best_greedy, traveling_times, inspection_times, open_hours)
print('Random state ---> {}'.format(routes))
print("Objective function's values ---> {}".format(-score))
map = display_routes(routes)
map

Random state ---> [[0, 19, 5, 12, 6, 1, 3, 14, 7, 20, 15, 9, 0], [0, 17, 18, 8, 16, 2, 13, 4, 11, 10, 0]]
Objective function's values ---> 175714.7


In [56]:
best_greedy = [1, 2, 1, 2, 1, 1, 1, 2, 1, 2, 2, 1, 2, 1, 1, 2, 2, 2, 1, 1]
best_a_star = [1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 2, 2, 2, 1, 2, 1, 1, 1, 1, 1]
score, routes = objective_function_a_star(best_a_star, traveling_times, inspection_times, open_hours)
print('Random state ---> {}'.format(routes))
print("Objective function's values ---> {}".format(-score))
map = display_routes(routes)
map

Random state ---> [[0, 19, 17, 5, 2, 6, 16, 4, 1, 14, 20, 7, 18, 3, 9, 0], [0, 13, 11, 15, 10, 12, 8, 0]]
Objective function's values ---> 111212.80000000002


# Simulated Annealing

In [11]:
# Neighborhood definition

def get_neighbor_solution1(solution):
    neighbor = copy.deepcopy(solution)
    establishment_to_mutate = np.random.randint(0, num_establishments)
    neighbor[establishment_to_mutate] = (neighbor[establishment_to_mutate] + np.random.randint(1, vehicles)-1) % vehicles + 1
    return neighbor

# Exchange the vehicles of two establishments
def get_neighbor_solution2(solution):
    neighbor_solution = copy.deepcopy(solution)
    establishment1 = np.random.randint(0, num_establishments)
    establishment2 = (establishment1 + np.random.randint(1, num_establishments)) % num_establishments
    neighbor_solution[establishment1], neighbor_solution[establishment2] = neighbor_solution[establishment2], neighbor_solution[establishment1]
    return neighbor_solution

# Neighbour 1 or 2 with 50% each
def get_neighbor_solution3(solution):
    if (np.random.randint(0,2)==0):
        return get_neighbor_solution1(solution)
    else:
        return get_neighbor_solution2(solution)

In [12]:
#Test Neighbours
s = generate_random_solution()
print(s)
for d1 in range(0,20):
    print('Neighbor number {} with get_neighbor_solution1 ---> {}'.format(d1+1, get_neighbor_solution1(s)))

for d1 in range(0,20):
    print('Neighbor number {} with get_neighbor_solution2 ---> {}'.format(d1+1, get_neighbor_solution2(s)))

[1 1 2 1 2 2 1 2 2 1 1 2 1 2 1 2 1 2 2 1]
Neighbor number 1 with get_neighbor_solution1 ---> [1 1 2 1 2 2 1 2 2 1 1 1 1 2 1 2 1 2 2 1]
Neighbor number 2 with get_neighbor_solution1 ---> [1 1 2 1 2 2 1 2 2 1 1 2 1 2 1 2 1 1 2 1]
Neighbor number 3 with get_neighbor_solution1 ---> [1 1 2 1 2 2 1 2 2 1 1 2 1 2 1 2 2 2 2 1]
Neighbor number 4 with get_neighbor_solution1 ---> [1 1 2 1 2 2 1 2 2 1 1 2 1 1 1 2 1 2 2 1]
Neighbor number 5 with get_neighbor_solution1 ---> [1 1 2 1 2 2 1 2 2 1 1 2 1 2 1 2 1 2 1 1]
Neighbor number 6 with get_neighbor_solution1 ---> [1 1 2 1 2 2 1 1 2 1 1 2 1 2 1 2 1 2 2 1]
Neighbor number 7 with get_neighbor_solution1 ---> [1 1 2 1 1 2 1 2 2 1 1 2 1 2 1 2 1 2 2 1]
Neighbor number 8 with get_neighbor_solution1 ---> [1 1 2 1 2 2 1 2 2 2 1 2 1 2 1 2 1 2 2 1]
Neighbor number 9 with get_neighbor_solution1 ---> [1 1 2 1 2 2 1 2 2 1 1 2 1 2 1 2 1 1 2 1]
Neighbor number 10 with get_neighbor_solution1 ---> [1 1 2 1 2 2 1 2 2 1 1 2 1 2 1 2 1 2 2 2]
Neighbor number 11 with get

In [13]:
def get_sa_solution(num_iterations, neighbor_operator, traveling_times, inspection_times, open_hours, objective_func,log=False):
    iteration = 0;
    temperature = 1000;
    solution = generate_random_solution() # Best solution after 'num_iterations' iterations without improvement
    score, routes = objective_func(solution, traveling_times, inspection_times, open_hours)
    
    best_solution = copy.deepcopy(solution)
    best_score = score
    best_routes = routes
    
    print(f"Init Solution:  {best_solution}, score: {-best_score}")
    
    while iteration < num_iterations:
        temperature = temperature * 0.9999  # Test with different cooling schedules
        iteration += 1
        neighbor_solution = neighbor_operator(best_solution)  #Test with Neighbour 1, 2 and 3
        neighbor_score, neighbor_routes = objective_func(neighbor_solution, traveling_times, inspection_times, open_hours)
        
        delta = neighbor_score - score 
        
        if delta > 0 or np.exp(-delta/temperature) > random.random():
            solution = neighbor_solution
            score = neighbor_score
            routes = neighbor_routes
            if score > best_score:
                iteration = 0
                best_solution = copy.deepcopy(solution)
                best_score = score
                best_routes = routes
                if log:
                    print(f"Solution: score: {-best_score},  Temp: {temperature}")
                
    print(f"Final Solution: {best_solution}, score: {-best_score}")
    return best_solution, best_routes

In [62]:
print("\nSimulated Annealing:\n")
solution, routes = get_sa_solution(2000, get_neighbor_solution1, traveling_times, inspection_times, open_hours, objective_function, True)


Simulated Annealing:

Init Solution:  [2 2 2 1 1 2 2 2 1 1 1 1 1 1 1 1 2 1 2 2], score: 211397.6
Solution: score: 183625.0,  Temp: 969.4605362958226
Solution: score: 180350.5,  Temp: 962.6946373158061
Solution: score: 180166.5,  Temp: 958.8496310845508
Solution: score: 160361.6,  Temp: 955.0199818235594
Final Solution: [2 1 1 1 1 2 2 2 1 1 1 1 1 1 1 1 2 1 2 2], score: 160361.6


In [17]:
print("\nSimulated Annealing:\n")
solution, routes = get_sa_solution(2000, get_neighbor_solution1, traveling_times, inspection_times, open_hours, objective_function_a_star, True)


Simulated Annealing:

Init Solution:  [ 12  57  44  65  87  37  44  34  58  44 100  19  29  62  21  62  61  74
  66  94  97  29  36  50  87  88  42  80  45  22  98  66  94   8  23  61
  57  40  78  64  88  94  90  99  83 100  97  80  30  20  22   3  63  26
   7  41  38  13  34  67  45  13  49  85   9  79   6  14  90  40  99  82
   9  16  93  97  49  13  10  15  45  24  30   9  88  59  35  33  84   3
  39   6  75  13  35  60  41  98  23  88  26  85  79  81  13  62   3  66
  95  16   7  92  46  63  97  29  95  92  54  59  86  15  77  90  41  31
  51  75  62  62  20  72  72  14  47   3  94  19  32  61  94  67  11  19
   3  46  60  69  30   5  46   7  94  21  67  55  87  92  67  99  10  59
  60  39  82  48  13  23  41  53  83  86  69  61  63  22  46  81  57  25
  26  41  77  98  52   1  73  25  31  41  45  23  99   7  68  93  51  70
  65  21  79   2  17  53  76  86  70  91   5  72  81  48  78  64  49  40
  18  51  15  28  45  72  26  29  86  70  17  12  52  61  84  46  25  25
  46  91  47

KeyboardInterrupt: 

In [64]:
best_greedy = [2, 1, 1, 1, 1, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 2]
best_a_star = [2, 2, 1, 2, 1, 1, 1, 1, 2, 2, 1, 1, 2, 1, 2, 2, 1, 1, 1, 1]
score, routes = objective_function(best_greedy, traveling_times, inspection_times, open_hours)
print('Random state ---> {}'.format(routes))
print("Objective function's values ---> {}".format(-score))
map = display_routes(routes)
map

Random state ---> [[0, 5, 18, 12, 16, 2, 3, 13, 14, 4, 11, 10, 15, 9, 0], [0, 19, 17, 8, 6, 1, 7, 20, 0]]
Objective function's values ---> 160361.6


In [65]:
best_greedy = [2, 1, 1, 1, 1, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 2]
best_a_star = [2, 2, 1, 2, 1, 1, 1, 1, 2, 2, 1, 1, 2, 1, 2, 2, 1, 1, 1, 1]
score, routes = objective_function_a_star(best_a_star, traveling_times, inspection_times, open_hours)
print('Random state ---> {}'.format(routes))
print("Objective function's values ---> {}".format(-score))
map = display_routes(routes)
map

Random state ---> [[0, 19, 17, 5, 6, 14, 11, 20, 7, 12, 8, 18, 3, 0], [0, 16, 2, 1, 13, 4, 15, 10, 9, 0]]
Objective function's values ---> 108482.90000000001


# Tabu Search

In [14]:
def get_tabu_search_solution(num_iterations, tabu_list_length, neighbor_operator, traveling_times, inspection_times, open_hours, objective_func,log=False):
    current_solution = generate_random_solution()
    current_score, current_route = objective_func(current_solution, traveling_times, inspection_times, open_hours)
    
    best_solution = current_solution
    best_score = current_score
    best_route = current_route
    
    tabu_list = []
    
    for iteration in range(num_iterations):
        neighbor_solutions = []
        neighbor_scores = []
        neighbor_routes = []
        tabu_neighbors = []
        
        for i in range(10):
            neighbor_solution = neighbor_operator(current_solution)
            neighbor_score, neighbor_route = objective_func(neighbor_solution, traveling_times, inspection_times, open_hours)
            if neighbor_solution.tolist() not in tabu_list:
                neighbor_solutions.append(neighbor_solution)
                neighbor_scores.append(neighbor_score)
                neighbor_routes.append(neighbor_route)
            else:
                tabu_neighbors.append(neighbor_solution)
        
        if len(neighbor_scores) > 0:
            best_neighbor_index = np.argmax(neighbor_scores)
            best_neighbor_score = neighbor_scores[best_neighbor_index]
            best_neighbor_solution = neighbor_solutions[best_neighbor_index]
            best_neighbor_route = neighbor_routes[best_neighbor_index]
            
            if best_neighbor_score > best_score:
                best_solution = best_neighbor_solution
                best_score = best_neighbor_score
                best_route = best_neighbor_route
            
            current_solution = best_neighbor_solution
            current_score = best_neighbor_score
            current_route = best_neighbor_route
            
            tabu_list.append(current_solution.tolist())
            if len(tabu_list) > tabu_list_length:
                tabu_list.pop(0)
                
        elif len(tabu_neighbors) > 0:
            best_tabu_index = np.argmax([objective_func(x, traveling_times, inspection_times, open_hours)[0] for x in tabu_neighbors])
            best_tabu_solution = tabu_neighbors[best_tabu_index]
            
            current_solution = best_tabu_solution
            current_score, current_route = objective_func(current_solution, traveling_times, inspection_times, open_hours)
            
            tabu_list.append(current_solution.tolist())
            if len(tabu_list) > tabu_list_length:
                tabu_list.pop(0)
        
        if log:
            print(f"Iteration {iteration}, Best Score: {-best_score}")
    
    print(f"Best solution: {best_solution}, Best Score: {-best_score}, Routes: {best_route}")
    
    return best_solution, best_route

In [14]:
print("\nTabu Search:\n")
solution, routes = get_tabu_search_solution(500000, 50000, get_neighbor_solution3, traveling_times, inspection_times, open_hours, objective_function, True)


Tabu Search:

Iteration 0, Best Score: 11274137.400000008
Iteration 1, Best Score: 11245337.400000008
Iteration 2, Best Score: 11175810.800000006
Iteration 3, Best Score: 11134000.400000004


KeyboardInterrupt: 

In [16]:
print("\nTabu Search:\n")
solution, routes = get_tabu_search_solution(1000, 50, get_neighbor_solution3, traveling_times, inspection_times, open_hours, objective_function_a_star, True)


Tabu Search:

Iteration 0, Best Score: 159657.9
Iteration 1, Best Score: 128934.79999999999
Iteration 2, Best Score: 122917.59999999999
Iteration 3, Best Score: 122531.0
Iteration 4, Best Score: 116542.1
Iteration 5, Best Score: 115126.20000000001
Iteration 6, Best Score: 115126.20000000001
Iteration 7, Best Score: 114982.09999999999
Iteration 8, Best Score: 112808.5
Iteration 9, Best Score: 112808.5
Iteration 10, Best Score: 111866.0
Iteration 11, Best Score: 111677.9
Iteration 12, Best Score: 108184.40000000001
Iteration 13, Best Score: 108184.40000000001
Iteration 14, Best Score: 108184.40000000001
Iteration 15, Best Score: 106603.6
Iteration 16, Best Score: 106603.6
Iteration 17, Best Score: 106603.6
Iteration 18, Best Score: 106603.6
Iteration 19, Best Score: 106603.6
Iteration 20, Best Score: 106603.6
Iteration 21, Best Score: 106603.6
Iteration 22, Best Score: 106603.6
Iteration 23, Best Score: 106603.6
Iteration 24, Best Score: 106603.6
Iteration 25, Best Score: 106603.6
Itera

Iteration 229, Best Score: 106603.6
Iteration 230, Best Score: 106603.6
Iteration 231, Best Score: 106603.6
Iteration 232, Best Score: 106603.6
Iteration 233, Best Score: 106603.6
Iteration 234, Best Score: 106603.6
Iteration 235, Best Score: 106603.6
Iteration 236, Best Score: 106603.6
Iteration 237, Best Score: 106603.6
Iteration 238, Best Score: 106603.6
Iteration 239, Best Score: 106603.6
Iteration 240, Best Score: 106603.6
Iteration 241, Best Score: 106603.6
Iteration 242, Best Score: 106603.6
Iteration 243, Best Score: 106603.6
Iteration 244, Best Score: 106603.6
Iteration 245, Best Score: 106603.6
Iteration 246, Best Score: 106603.6
Iteration 247, Best Score: 106603.6
Iteration 248, Best Score: 106603.6
Iteration 249, Best Score: 106603.6
Iteration 250, Best Score: 106603.6
Iteration 251, Best Score: 106603.6
Iteration 252, Best Score: 106603.6
Iteration 253, Best Score: 106603.6
Iteration 254, Best Score: 106603.6
Iteration 255, Best Score: 106603.6
Iteration 256, Best Score: 1

Iteration 457, Best Score: 106603.6
Iteration 458, Best Score: 106603.6
Iteration 459, Best Score: 106603.6
Iteration 460, Best Score: 106603.6
Iteration 461, Best Score: 106603.6
Iteration 462, Best Score: 106603.6
Iteration 463, Best Score: 106603.6
Iteration 464, Best Score: 106603.6
Iteration 465, Best Score: 106603.6
Iteration 466, Best Score: 106603.6
Iteration 467, Best Score: 106603.6
Iteration 468, Best Score: 106603.6
Iteration 469, Best Score: 106603.6
Iteration 470, Best Score: 106603.6
Iteration 471, Best Score: 106603.6
Iteration 472, Best Score: 106603.6
Iteration 473, Best Score: 106603.6
Iteration 474, Best Score: 106603.6
Iteration 475, Best Score: 106603.6
Iteration 476, Best Score: 106603.6
Iteration 477, Best Score: 106603.6
Iteration 478, Best Score: 106603.6
Iteration 479, Best Score: 106603.6
Iteration 480, Best Score: 106603.6
Iteration 481, Best Score: 106603.6
Iteration 482, Best Score: 106603.6
Iteration 483, Best Score: 106603.6
Iteration 484, Best Score: 1

Iteration 686, Best Score: 106603.6
Iteration 687, Best Score: 106603.6
Iteration 688, Best Score: 106603.6
Iteration 689, Best Score: 106603.6
Iteration 690, Best Score: 106603.6
Iteration 691, Best Score: 106603.6
Iteration 692, Best Score: 106603.6
Iteration 693, Best Score: 106603.6
Iteration 694, Best Score: 106603.6
Iteration 695, Best Score: 106603.6
Iteration 696, Best Score: 106603.6
Iteration 697, Best Score: 106603.6
Iteration 698, Best Score: 106603.6
Iteration 699, Best Score: 106603.6
Iteration 700, Best Score: 106603.6
Iteration 701, Best Score: 106603.6
Iteration 702, Best Score: 106603.6
Iteration 703, Best Score: 106603.6
Iteration 704, Best Score: 106603.6
Iteration 705, Best Score: 106603.6
Iteration 706, Best Score: 106603.6
Iteration 707, Best Score: 106603.6
Iteration 708, Best Score: 106603.6
Iteration 709, Best Score: 106603.6
Iteration 710, Best Score: 106603.6
Iteration 711, Best Score: 106603.6
Iteration 712, Best Score: 106603.6
Iteration 713, Best Score: 1

Iteration 915, Best Score: 106603.6
Iteration 916, Best Score: 106603.6
Iteration 917, Best Score: 106603.6
Iteration 918, Best Score: 106603.6
Iteration 919, Best Score: 106603.6
Iteration 920, Best Score: 106603.6
Iteration 921, Best Score: 106603.6
Iteration 922, Best Score: 106603.6
Iteration 923, Best Score: 106603.6
Iteration 924, Best Score: 106603.6
Iteration 925, Best Score: 106603.6
Iteration 926, Best Score: 106603.6
Iteration 927, Best Score: 106603.6
Iteration 928, Best Score: 106603.6
Iteration 929, Best Score: 106603.6
Iteration 930, Best Score: 106603.6
Iteration 931, Best Score: 106603.6
Iteration 932, Best Score: 106603.6
Iteration 933, Best Score: 106603.6
Iteration 934, Best Score: 106603.6
Iteration 935, Best Score: 106603.6
Iteration 936, Best Score: 106603.6
Iteration 937, Best Score: 106603.6
Iteration 938, Best Score: 106603.6
Iteration 939, Best Score: 106603.6
Iteration 940, Best Score: 106603.6
Iteration 941, Best Score: 106603.6
Iteration 942, Best Score: 1

In [17]:
#best_greedy = [2, 1, 1, 1, 2, 2, 2, 1, 1, 1, 1, 2, 2, 1, 1, 2, 1, 1, 2, 2]
#best_a_star = [1, 1, 1, 1, 2, 2, 2, 1, 1, 1, 1, 2, 1, 1, 1, 2, 1, 2, 2, 2]
#score, routes = objective_function(best_greedy, traveling_times, inspection_times, open_hours)
print('Random state ---> {}'.format(routes))
print("Objective function's values ---> {}".format(-solution))
map = display_routes(routes)
map

Random state ---> [[0, 17, 8, 2, 14, 1, 3, 13, 4, 11, 15, 10, 9, 0], [0, 19, 5, 6, 16, 20, 7, 12, 18, 0]]
Objective function's values ---> [-1 -1 -1 -1 -2 -2 -2 -1 -1 -1 -1 -2 -1 -1 -1 -2 -1 -2 -2 -2]


In [81]:
best_greedy = [2, 1, 1, 1, 2, 2, 2, 1, 1, 1, 1, 2, 2, 1, 1, 2, 1, 1, 2, 2]
best_a_star = [1, 1, 1, 1, 2, 2, 2, 1, 1, 1, 1, 2, 1, 1, 1, 2, 1, 2, 2, 2]
score, routes = objective_function_a_star(best_a_star, traveling_times, inspection_times, open_hours)
print('Random state ---> {}'.format(routes))
print("Objective function's values ---> {}".format(-score))
map = display_routes(routes)
map

Random state ---> [[0, 17, 8, 2, 14, 1, 3, 13, 4, 11, 15, 10, 9, 0], [0, 19, 5, 6, 16, 20, 7, 12, 18, 0]]
Objective function's values ---> 106603.6


# Hill-Climbing

In [84]:
def get_hc_solution(num_iterations, neighbor_operator, traveling_times, inspection_times, open_hours, objective_func, log=False):
    iteration = 0;
    best_solution = generate_random_solution() # Best solution after 'num_iterations' iterations without improvement
    best_score, best_routes = objective_func(best_solution, traveling_times, inspection_times, open_hours)
    
    print(f"Init Solution: score: {-best_score}")
    
    while iteration < num_iterations:
        iteration += 1
        neighbor_solution = neighbor_operator(best_solution)   #Test with Neighbour 1, 2 and 3
        neighbor_score, neighbor_routes = objective_func(neighbor_solution, traveling_times, inspection_times, open_hours)
        if neighbor_score > best_score:
            iteration = 0
            best_solution = neighbor_solution
            best_score = neighbor_score
            best_routes = neighbor_routes
            if log:
                (print(f"Solution {iteration}: score: {-best_score}"))
            
    print(f"Final Solution: solution: {best_solution}, score: {-best_score}, routes: {best_routes}")
    return best_solution, best_routes

In [86]:
print("Hill climbing:\n")
solution, routes = get_hc_solution(2000, get_neighbor_solution3, traveling_times, inspection_times, open_hours, objective_function, True)

Hill climbing:

Init Solution: score: 174179.40000000002
Solution 0: score: 173476.1
Solution 0: score: 155452.5
Solution 0: score: 153530.90000000002
Solution 0: score: 149474.5
Solution 0: score: 128726.6
Final Solution: solution: [1 2 1 2 1 1 1 2 2 2 2 1 1 1 2 1 1 2 1 2], score: 128726.6, routes: [[0, 19, 17, 5, 12, 6, 16, 1, 3, 13, 14, 7, 0], [0, 18, 8, 2, 4, 11, 20, 10, 15, 9, 0]]


In [87]:
print("Hill climbing:\n")
solution, routes = get_hc_solution(2000, get_neighbor_solution3, traveling_times, inspection_times, open_hours, objective_function_a_star, True)

Hill climbing:

Init Solution: score: 176134.6
Solution 0: score: 148627.40000000002
Solution 0: score: 123451.3
Solution 0: score: 120608.30000000002
Solution 0: score: 118596.20000000001
Solution 0: score: 114514.9
Solution 0: score: 114105.2
Solution 0: score: 112161.5
Solution 0: score: 111907.0
Solution 0: score: 108184.40000000001
Solution 0: score: 106920.00000000001
Solution 0: score: 106603.6
Final Solution: solution: [2 2 2 2 1 1 1 2 2 2 2 1 2 2 2 1 2 1 1 1], score: 106603.6, routes: [[0, 19, 5, 6, 16, 20, 7, 12, 18, 0], [0, 17, 8, 2, 14, 1, 3, 13, 4, 11, 15, 10, 9, 0]]


In [88]:
best_greedy = [1, 2, 1, 2, 1, 1, 1, 2, 2, 2, 2, 1, 1, 1, 2, 1, 1, 2, 1, 2]
best_a_star = [2, 2, 2, 2, 1, 1, 1, 2, 2, 2, 2, 1, 2, 2, 2, 1, 2, 1, 1, 1]
score, routes = objective_function(best_greedy, traveling_times, inspection_times, open_hours)
print('Random state ---> {}'.format(routes))
print("Objective function's values ---> {}".format(-score))
map = display_routes(routes)
map

Random state ---> [[0, 19, 17, 5, 12, 6, 16, 1, 3, 13, 14, 7, 0], [0, 18, 8, 2, 4, 11, 20, 10, 15, 9, 0]]
Objective function's values ---> 128726.6


In [90]:
best_greedy = [1, 2, 1, 2, 1, 1, 1, 2, 2, 2, 2, 1, 1, 1, 2, 1, 1, 2, 1, 2]
best_a_star = [2, 2, 2, 2, 1, 1, 1, 2, 2, 2, 2, 1, 2, 2, 2, 1, 2, 1, 1, 1]
score, routes = objective_function_a_star(best_a_star, traveling_times, inspection_times, open_hours)
print('Random state ---> {}'.format(routes))
print("Objective function's values ---> {}".format(-score))
map = display_routes(routes)
map

Random state ---> [[0, 19, 5, 6, 16, 20, 7, 12, 18, 0], [0, 17, 8, 2, 14, 1, 3, 13, 4, 11, 15, 10, 9, 0]]
Objective function's values ---> 106603.6


# Problem 2

In [ ]:
# convert the DataFrame to a dictionary
traveling_times = {}
for i in range(len(very_small_distances)):
    traveling_times[i] = very_small_distances.iloc[i].tolist()

num_establishments = len(traveling_times)-1
inspection_times = very_small_establishments[['Inspection Time']]
open_hours = very_small_establishments[['Opening Hours']]

def generate_random_solution():
    solution = np.zeros(num_establishments, dtype=int)
    values = list(range(1, vehicles + 1))
    np.random.shuffle(values)
    index = 0
    for i in range(num_establishments):
        solution[i] = values[index]
        index = (index + 1) % vehicles
    return solution


def objective_function2(chromosome, traveling_times, inspection_times, open_hours):
    total_travel_time = 0
    total_inspection_time = 0
    total_time_per_route = [0 for i in range(max(chromosome))]
    j = -1

    # Initialize an empty list to store the routes
    routes = [[] for i in range(max(chromosome))]

    # Assign each establishment to a route
    for i in range(len(chromosome)):
        routes[chromosome[i]-1].append(i+1)

    # Calculate the total travel time and inspection time for all routes
    for route in routes:
        if len(route) == 0:
            continue
        j += 1
        route_travel_time = 0
        route_inspection_time = 0
        current_time = 0
        # Add the departure/arrival establishment to the beginning and end of the route
        route = [0] + route
        
        # Use a greedy search algorithm to find the optimal path for the current route
        route = greedy_search(route, heuristic_distances)
        
        start_time = ast.literal_eval(open_hours.iloc[route[1]][0]).index(1)  # Inspection start time of first establishment in route
        #print('start time: {}'.format(start_time))

        for i in range(1, len(route)):
            # Calculate traveling time between establishments
            current_establishment = route[i-1]
            next_establishment = route[i]
            travel_time = traveling_times[current_establishment][next_establishment]
            #route_travel_time += traveling_times[current_establishment][next_establishment]
            
            # Calculate inspection time and waiting time based on establishment's schedule
            current_inspection_time = 5*60 # Convert minutes to seconds
            current_open_hours = ast.literal_eval(open_hours.iloc[next_establishment][0])
            
            # Calculate waiting time if establishment is closed
            if i == 1:
                waiting_time = 0
            else:
                current_hour = int((current_time + travel_time + (start_time*3600))/3600)
                #print('Current hour: {}'.format(current_hour))
                while current_open_hours[current_hour % 24] == 0:
                    current_hour += 1
                waiting_time = (current_hour * 3600) - (current_time + travel_time + (start_time*3600))
                #print('({} * 3600) - ({} + {} + ({}*3600))'.format(current_hour, current_time, travel_time, start_time))
                
            # Add inspection time and waiting time to current time
            if i > 1:
                current_time += max(waiting_time, 0)
            current_time += travel_time
            current_time += current_inspection_time
            
            if current_time > 28800:
                return -1000000000000
            
            #print('Iteration {} ---> current establishment: {}, next establishment: {}, inspection time: {}, open hours array: {}, travel time: {}, waiting time: {}, current time: {}'.format(i, current_establishment, next_establishment, current_inspection_time, current_open_hours, travel_time, waiting_time, current_time))
            # Reset waiting time
            waiting_time = 0
            
            # Add inspection time and traveling time to route times
            #route_inspection_time += current_inspection_time
            route_travel_time = 0

        # Add total time for current route to total time per route
        total_time_per_route[j] = current_time
        #print('total time per route ----> {}'.format(total_time_per_route))

    # Add total time for all routes to total travel time
    total_travel_time += sum(total_time_per_route)

def objective_function_a_star2(chromosome, traveling_times, inspection_times, open_hours):
    
    total_travel_time = 0
    l = 1
    
    # Initialize an empty list to store the routes
    routes = [[] for i in range(max(chromosome))]
    
    # Assign each establishment to a route
    for i in range(len(chromosome)):
        routes[chromosome[i]-1].append(i+1)
    
    # Calculate the total travel time for all routes
    for route in routes:
        if len(route) == 0:
            continue
        j = 1
        current_time = 0
        
        # Create a set with all establishments in the route
        route_establishments = set(route)
        route_len = len(route_establishments)
        
        # Add the depot to the beginning of the route
        route = [0]
        
        # Initialize the route travel time as zero
        route_travel_time = 0
        
        while route_establishments:
            # Calculate the A* algorithm for the next establishment
            current_establishment = route[-1]
            next_establishment = None
            best_score = float('inf')
            for e in route_establishments:
                i = 1
                # Calculate the total time (travel time, inspection time, waiting time, heuristic value) for the next establishment
                travel_time = traveling_times[route[-1]][e]
                inspection_time = 5 * 60
                current_open_hours = ast.literal_eval(open_hours.iloc[e][0])
                if len(route_establishments) == route_len:
                    start_time = current_open_hours.index(1)
                i += 1
                # Calculate waiting time if establishment is closed
                if len(route) == 1:
                    waiting_time = 0
                else:
                    current_hour = int((current_time + travel_time + (start_time*3600))/3600)
                    while current_open_hours[current_hour % 24] == 0:
                        current_hour += 1
                    waiting_time = (current_hour * 3600) - (current_time + travel_time + (start_time*3600))
                
                # Add inspection time, waiting time, and travel time to the current time
                time_to_next = max(waiting_time, 0) + inspection_time + travel_time
                total_time = current_time + time_to_next
                heuristic_value = heuristic_distances[e][0]
                score = total_time + heuristic_value
                
                # If the current establishment has the smallest score, update next_establishment
                if score < best_score:
                    '''
                    best_waiting_time = max(waiting_time, 0)
                    best_open_hours = current_open_hours
                    best_inspection_time = inspection_time
                    if len(route) == 1:
                        best_start_time = start_time
                    else:
                        best_current_hour = current_hour
                    best_current_time = total_time
                    best_travel_time = travel_time
                    '''
                    time_to_next_establishment = time_to_next
                    next_establishment = e
                    best_score = score
            '''
            if len(route) > 1:
                print('Current hour: {}'.format(best_current_hour))
                print('({} * 3600) - ({} + {} + ({}*3600))'.format(best_current_hour, current_time, best_travel_time, best_start_time))
            else:
                print('start time: {}'.format(best_start_time))
            '''
            
            # Add next establishment to route and remove from set
            route_establishments.remove(next_establishment)
            route.append(next_establishment)
            
            #print('Update current time: {} + {}'.format(current_time, time_to_next_establishment))
            # Update the current time and route travel time
            current_time += time_to_next_establishment
            #route_travel_time += travel_time
            #print('Iteration {} ---> current establishment: {}, next establishment: {}, next inspection time: {}, open hours array: {}, travel time: {}, waiting time: {}, current time: {}'.format(j, current_establishment, next_establishment, best_inspection_time, best_open_hours, best_travel_time, best_waiting_time, best_current_time))
        
            if current_time > 28800:
                return -1000000000000
        
            j += 1

        #print('Route: {}'.format(route))
        # Calculate the total time for the current route and add to total time per route
        total_time_per_route = current_time
        #print('Traveling time back to the depot: {}'.format(traveling_times[route[-2]][0]))
        #print('Total travel time in route {}: {}'.format(l, total_time_per_route))
        total_travel_time += total_time_per_route
        l += 1
        
    return -total_travel_time # Minimize the total travel time

In [ ]:
vehicles = 1

while True:
    print("\nSimulated Annealing:")
    print(f"Number of vehicles: {vehicles}\n")
    best_solution = get_sa_solution(5000, get_neighbor_solution2, traveling_times, inspection_times, open_hours, objective_function_a_star2,True)
    best_score = objective_function_a_star2(best_solution, traveling_times, inspection_times, open_hours)
    if best_score > -10000000:
        print(f"Final Solution: {best_solution}, score: {-best_score}, num_vehicles: {vehicles}")
        break
    vehicles += 1

In [ ]:
vehicles = 1

while True:
    print("\nTabu Search:")
    print(f"Number of vehicles: {vehicles}\n")
    best_solution = get_tabu_search_solution(500, 40, get_neighbor_solution2, traveling_times, inspection_times, open_hours, objective_function_a_star2, True)
    best_score = objective_function_a_star2(best_solution, traveling_times, inspection_times, open_hours)
    if best_score > -10000000:
        print(f"Final Solution: {best_solution}, score: {-best_score}, num_vehicles: {vehicles}")
        break
    vehicles += 1

In [ ]:
vehicles= 9

while True:
    vehicles += 1
    print("\nHill climbing:")
    print(f"Number of vehicles: {vehicles}\n")
    best_solution = get_hc_solution(1000, get_neighbor_solution2, traveling_times, inspection_times, open_hours, objective_function2, True)
    best_score = objective_function2(best_solution, traveling_times, inspection_times, open_hours)
    if best_score > -10000000:
        print(f"Final Solution: {best_solution}, score: {-best_score}, num_vehicles: {vehicles}")
        break